<a href="https://colab.research.google.com/github/aadi-kanwar/Gen-AI/blob/main/Exp_4_DC_GAN_Celeb_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DC GAN using CelebA Dataset

### Import Libraries & Dependencies

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import zipfile
from glob import glob
from PIL import Image
from google.colab import drive

In [ ]:
# Define paths
zip_path = '/content/drive/MyDrive/Colab Notebooks/Datasets/Celeb/img_align_celeba.zip'
extract_path = '/content/celeba_images'

In [ ]:
# Extract images if not already extracted
if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

image_folder = os.path.join(extract_path, 'img_align_celeba')
image_paths = glob(os.path.join(image_folder, '*.jpg'))
image_paths = image_paths[:5000]

In [ ]:
# Load and preprocess images
def load_image(image_path):
    img = Image.open(image_path).resize((64, 64))
    img = np.array(img).astype(np.float32)
    img = (img - 127.5) / 127.5  # Normalize to [-1, 1]
    return img

In [ ]:
BUFFER_SIZE = len(image_paths)
BATCH_SIZE = 64

In [ ]:
train_images = np.array([load_image(img) for img in image_paths])
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
# Generator model
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(8*8*256, use_bias=False, input_shape=(100,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Reshape((8, 8, 256)),
        layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model

generator = build_generator()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:

# Discriminator model
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[64, 64, 3]),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Flatten(),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

discriminator = build_discriminator()
discriminator.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = tf.keras.Sequential([generator, discriminator])
    return model

gan = build_gan(generator, discriminator)
gan.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss='binary_crossentropy')

In [ ]:
# Training function
EPOCHS = 5
noise_dim = 100
num_examples_to_generate = 16
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        # Calculate losses for generator and discriminator
        gen_loss = tf.keras.losses.binary_crossentropy(tf.ones_like(fake_output), fake_output)
        disc_loss = tf.keras.losses.binary_crossentropy(tf.ones_like(real_output), real_output) + \
                    tf.keras.losses.binary_crossentropy(tf.zeros_like(fake_output), fake_output)

    # Calculate gradients for generator and discriminator
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    # Apply gradients if they exist for the discriminator
    if gradients_of_discriminator:  # Check if gradients_of_discriminator is not empty
        discriminator.optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    generator.optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

In [ ]:
def train(dataset, epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            train_step(image_batch)
        print(f'Epoch {epoch+1}/{epochs} completed')
        generate_and_save_images(generator, epoch + 1, seed)

In [ ]:
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow((predictions[i] + 1) / 2)
        plt.axis('off')
    plt.savefig(f'image_at_epoch_{epoch}.png')
    plt.show()

In [ ]:
# Train the model
generator.optimizer = tf.keras.optimizers.Adam(1e-4)
train(train_dataset, EPOCHS)

InvalidArgumentError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes: [8] vs. [64] [Op:AddV2] name: 